In [1]:
#Import the gtfs-kit module
import gtfs_kit as gk
import pandas as pd


In [2]:
feeds = []
feed1 = gk.feed.read_feed('../20161210.gtfs',dist_units="km")
feed2 = gk.feed.read_feed('../20170415.gtfs',dist_units="km")
feed3 = gk.feed.read_feed('../20171224.gtfs',dist_units="km")
feed4 = gk.feed.read_feed('../20181211.gtfs',dist_units="km")
feed5 = gk.feed.read_feed('../20191229.gtfs',dist_units="km")
feed6 = gk.feed.read_feed('../20201217.gtfs',dist_units="km")
feed7 = gk.feed.read_feed('../20211214.gtfs',dist_units="km")
feed8 = gk.feed.read_feed('../20221214.gtfs',dist_units="km")
print(feed1)
feeds.extend((feed1,feed2,feed3,feed4,feed5,feed6,feed7,feed8))

* agency --------------------
	  agency_id        agency_name            agency_url agency_timezone   
0       163  TRENITALIA S.p.A.  http://trenitalia.it     Europe/Rome  \

  agency_lang  
0          it  
* attributions --------------------
	None
* calendar --------------------
	None
* calendar_dates --------------------
	    service_id      date  exception_type
0  1740_167801  20160919               1
1  1740_167801  20160920               1
2  1740_167801  20160921               1
3  1740_167801  20160922               1
4  1740_167801  20160923               1
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	None
* frequencies --------------------
	None
* routes --------------------
	     route_id agency_id route_short_name   
0  1004295345       163              NaN  \
1  1013468805       163              NaN   
2  1017917198       163              NaN   
3  1023125973       163              NaN   
4  10389098

In [3]:
dataset = pd.DataFrame()
y=0
for feed in feeds:
    df = pd.DataFrame()
    routes = gk.routes.get_routes(feed)
    stops_time = feed.get_stop_times()
    trips = feed.get_trips()
    stops = feed.get_stops()
    calendar_dates = feed.calendar_dates
    ### Get the route_id for Pisa-Lucca-Aulla
    sub_df = routes.loc[routes['route_long_name'] == 'Pisa - Lucca - Aulla'].reset_index(drop=True)
    route_id = sub_df.iloc[0]['route_id']
    ### Get all the trips in for Pisa-Lucca-Aulla
    trips = trips.loc[trips['route_id'] == route_id]
    ### Merge between routes and trips
    df = pd.merge(routes, trips, on=['route_id','route_id'])
    ### merge between (routes,trips) and stop_times
    df = pd.merge(df, stops_time, on=['trip_id','trip_id'])
    ### merge between (routes,trips,stops_times) and stop
    df = pd.merge(df, stops, on=['stop_id','stop_id'])
    ### Merge betwenn (routes,trips,stops_times,stop) and calendar_dates
    df = pd.merge(df, calendar_dates, on=['service_id','service_id'])
    ### Drop not interested columns
    df = df.drop(['agency_id', 'route_short_name', 'route_type', 'route_color', 'route_text_color','shape_id','shape_dist_traveled'], axis=1)
    dataset = pd.concat([dataset, df])


In [4]:
dataset = dataset.reset_index(drop=True)
print(dataset)
dataset.to_csv('dataset.csv')

           route_id       route_long_name   service_id      trip_id   
0        1085727347  Pisa - Lucca - Aulla  1814_171965  1814_191301  \
1        1085727347  Pisa - Lucca - Aulla  1814_171965  1814_191301   
2        1085727347  Pisa - Lucca - Aulla  1814_171965  1814_191301   
3        1085727347  Pisa - Lucca - Aulla  1814_171965  1814_191301   
4        1085727347  Pisa - Lucca - Aulla  1814_171965  1814_191301   
...             ...                   ...          ...          ...   
1718432  1199544473  Pisa - Lucca - Aulla  4118_237648  4118_296638   
1718433  1199544473  Pisa - Lucca - Aulla  4118_237648  4118_296638   
1718434  1199544473  Pisa - Lucca - Aulla  4118_237648  4118_296638   
1718435  1199544473  Pisa - Lucca - Aulla  4118_237648  4118_296638   
1718436  1199544473  Pisa - Lucca - Aulla  4118_237648  4118_296638   

        trip_headsign trip_short_name  direction_id arrival_time   
0           Pisa C.Le           06772             1     15:42:00  \
1          